In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np


In [2]:
# Define transformation pipeline for preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image or numpy.ndarray to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1] range
])

batch_size = 32  # Reduce batch size due to memory issues

# Load CIFAR-10 dataset and create data loaders
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Define classes in CIFAR-10 dataset
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Function to display images
def display_images(images, labels):
    for j in range(len(images)):
        plt.figure(figsize=(5, 5))
        image = images[j] / 2 + 0.5  # Unnormalize
        npimg = image.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.title(f'Ground Truth: {classes[labels[j]]}')
        plt.axis('off')
        plt.show()


In [6]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 3 input channels (RGB), 6 output channels, 5x5 kernel
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling with 2x2 kernel and stride
        self.conv2 = nn.Conv2d(6, 16, 5)  # 6 input channels, 16 output channels, 5x5 kernel
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # Fully connected layer: 16 * 5 * 5 input features, 120 output features
        self.fc2 = nn.Linear(120, 84)  # Fully connected layer: 120 input features, 84 output features
        self.fc3 = nn.Linear(84, 10)  # Fully connected layer: 84 input features, 10 output features (classes)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))  # Convolution -> ReLU -> Max pooling
        x = self.pool(nn.functional.relu(self.conv2(x)))  # Convolution -> ReLU -> Max pooling
        x = torch.flatten(x, 1)  # Flatten 2D features to 1D
        x = nn.functional.relu(self.fc1(x))  # Fully connected -> ReLU
        x = nn.functional.relu(self.fc2(x))  # Fully connected -> ReLU
        x = self.fc3(x)  # Output layer
        return x

# Create an instance of the CNN model
model = CNN()


In [7]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross entropy loss function for multi-class classification
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # SGD optimizer with learning rate and momentum


In [8]:
# Training loop
epochs = 10  # Number of training epochs
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data  # Get the inputs
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize
        running_loss += loss.item()  # Accumulate loss
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')


[1,   100] loss: 2.303
[1,   200] loss: 2.302
[1,   300] loss: 2.300
[1,   400] loss: 2.298
[1,   500] loss: 2.296
[1,   600] loss: 2.291
[1,   700] loss: 2.285
[1,   800] loss: 2.271
[1,   900] loss: 2.250
[1,  1000] loss: 2.184
[1,  1100] loss: 2.111
[1,  1200] loss: 2.042
[1,  1300] loss: 2.015
[1,  1400] loss: 1.989
[1,  1500] loss: 1.963
[2,   100] loss: 1.921
[2,   200] loss: 1.919
[2,   300] loss: 1.915
[2,   400] loss: 1.859
[2,   500] loss: 1.843
[2,   600] loss: 1.825
[2,   700] loss: 1.796
[2,   800] loss: 1.765
[2,   900] loss: 1.770
[2,  1000] loss: 1.741
[2,  1100] loss: 1.712
[2,  1200] loss: 1.687
[2,  1300] loss: 1.691
[2,  1400] loss: 1.651
[2,  1500] loss: 1.655
[3,   100] loss: 1.616
[3,   200] loss: 1.603
[3,   300] loss: 1.596
[3,   400] loss: 1.609
[3,   500] loss: 1.587
[3,   600] loss: 1.558
[3,   700] loss: 1.552
[3,   800] loss: 1.555
[3,   900] loss: 1.541
[3,  1000] loss: 1.535
[3,  1100] loss: 1.537
[3,  1200] loss: 1.559
[3,  1300] loss: 1.546
[3,  1400] 

In [10]:
# Test the model on the test dataset
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {total} test images: {100 * correct / total} %')


Accuracy of the network on the 10000 test images: 58.2 %


In [12]:
# Calculate accuracy per class
class_correct = list(0. for _ in range(10))
class_total = list(0. for _ in range(10))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):  # Iterate over actual batch size
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# Print accuracy for each class
for i in range(10):
    if class_total[i] > 0:
        print(f'Accuracy of {classes[i]} : {100 * class_correct[i] / class_total[i]} %')
    else:
        print(f'Accuracy of {classes[i]} : N/A (no samples)')


Accuracy of plane : 61.8 %
Accuracy of car : 66.7 %
Accuracy of bird : 48.3 %
Accuracy of cat : 39.9 %
Accuracy of deer : 46.5 %
Accuracy of dog : 45.6 %
Accuracy of frog : 73.2 %
Accuracy of horse : 57.1 %
Accuracy of ship : 81.0 %
Accuracy of truck : 61.9 %
